# NLP PROJECT

Import

In [ ]:
from nltk.translate.metrics import *
import string
import numpy as np
from nltk import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
#nltk.download('punkt')
from nltk.tokenize import word_tokenize 
from sklearn.pipeline import Pipeline

Creating the dataframe 

In [ ]:
messages = pd.read_csv('smsspamcollection/SMSSpamCollection',sep='\t', names=['label', 'message'])

Use of stopwords

In [ ]:
 def rem_stopwords(text): 
    
  #  nopunc = [char for char in text if char not in string.punctuation]
# nopunc = ''.join(nopunc)
    
    stop_words = stopwords.words("english") + ['ok']
    word_tokens = word_tokenize(text) 
    
    filtered_text = [word for word in word_tokens if word.lower() not in stop_words] 
    
    return filtered_text 

In [ ]:
messages['message'].apply(rem_stopwords).head(6)

Vectorisation

In [ ]:
#Import 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [ ]:
# Create weighted matrix
boW = CountVectorizer(analyzer=rem_stopwords).fit(messages["message"])
#Sparse matrix 
message_bow = boW.transform(messages['message'])

TF-IDF

In [ ]:
#TF IDF
tfidf_trans = TfidfTransformer().fit(message_bow)
mess_tfidf = tfidf_trans.transform(message_bow)

Training model - Utilisation du naives bayes classifier 

In [ ]:
#Import
from sklearn.naive_bayes import MultinomialNB

# Modelisation

In [ ]:
from sklearn.model_selection  import train_test_split

In [ ]:
# training et test set 
mess_train,mess_test,label_train,label_test = train_test_split(messages['message'], messages['label'], test_size=0.4)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=rem_stopwords)),  # creation des tokens
    ('tfidf', TfidfTransformer()),  # calcul des poids tfifd 
    ('classifier', RandomForestClassifier()),  # utilisation du model
])


In [ ]:
pipeline.fit(mess_train,label_train) # training du model

In [ ]:
prediction = pipeline.predict(mess_test)

Mesure de performance

In [ ]:
from sklearn.metrics import *
from nltk.metrics.scores import *

In [ ]:
def performance(y_test,y_model):
    print ("accuracy", accuracy_score(y_test,y_model))
    print ("classification_report",classification_report(y_test,y_model))
    print ("F1 score",f1_score(y_test,y_model, average='macro'))
    #print ("AUC ROC",roc_auc_score(y_test,y_model))

In [ ]:
accuracy_score(label_test,prediction)

In [ ]:
performance(label_test,prediction)

In [ ]:
performance(label_test,prediction)

# GRIDSEARCH

In [ ]:
from sklearn.model_selection import GridSearchCV

param = { 
    'classifier__n_estimators' : [2,5,10,20,50],
    'classifier__max_features' : ['auto','sqrt', 'log2'],
    'classifier__max_depth' : [4,5,6,7,8,None],
    'classifier__criterion' :['gini', 'entropy'],
   # 'classifier__min_samples_split': [2, 5, 10, 15,20],
   # 'classifier__min_samples_leaf': [1, 2, 5, 10]   

}

grid = GridSearchCV(pipeline, cv=5, n_jobs=1, param_grid=param  ,scoring='accuracy')

In [ ]:
grid.fit(label_test,prediction) 
grid.best_params_

In [ ]:
grid.estimator.get_params().keys()

In [ ]:
best_grid = grid.best_estimator_
best_grid

In [ ]:
prediction_SV = best_grid.predict(mess_test)
performance(label_test,prediction)